In [4]:
from dash import Dash
from dash import dcc, html, Output, Input, State, callback_context
import plotly.graph_objects as go
import numpy as np

# ---------------------------
# Shared Functions and Constants
# ---------------------------
k = 8.625e-5        # Boltzmann constant (eV/K)
Ncv = 2.6e19        # Effective density of states (cm^-3)
ww = 2              # Band width for drawing (in eV)

def calculate_parameters(T, Eg, Na_log, Nd_log):
    Vt = k * T
    Na = 10**Na_log
    Nd = 10**Nd_log
    ni = Ncv * np.exp(-Eg/(2*Vt))
    if (Nd - Na) > 0:
        n = 0.5*(Nd - Na + np.sqrt((Nd - Na)**2 + 4*ni**2))
        p = ni**2/n
        Ef = 0.5*Eg - Vt*np.log(Ncv/n)
    elif (Nd - Na) < 0:
        p = 0.5*(Na - Nd + np.sqrt((Na - Nd)**2 + 4*ni**2))
        n = ni**2/p
        Ef = -0.5*Eg + Vt*np.log(Ncv/p)
    else:
        n = ni
        p = ni
        Ef = 0
    return Vt, Na, Nd, ni, n, p, Ef

def generate_scatter_points(n, p, aa, Vt):
    # Electrons
    if n > 2e7:
        Nel = int(np.round(((np.log10(n) - 10)**3)/7) + 3)
        Ekin_e = -5 * Vt * np.log(np.random.rand(Nel))
        xx_e = np.random.rand(Nel) * 2.8
        yy_e = aa + Ekin_e
    else:
        xx_e = np.array([])
        yy_e = np.array([])
    # Holes
    if p > 2e7:
        Nho = int(np.round(((np.log10(p) - 10)**3)/7) + 3)
        Ekin_h = -5 * Vt * np.log(np.random.rand(Nho))
        xx_h = np.random.rand(Nho) * 2.8
        yy_h = -aa - Ekin_h
    else:
        xx_h = np.array([])
        yy_h = np.array([])
    return xx_e, yy_e, xx_h, yy_h

# ---------------------------
# Define the Integrated App Layout with dcc.Store for y_max
# ---------------------------
app = Dash(__name__)
app.layout = html.Div([
    html.H2("Fermi-Dirac Simulation with Combined Energy Histogram"),
    
    # All three graphs in one row
    html.Div([
        dcc.Graph(id='graph_ex', config={'displayModeBar': False},
                  style={'width': '33%', 'display': 'inline-block'}),
        dcc.Graph(id='graph_ef', config={'displayModeBar': False},
                  style={'width': '33%', 'display': 'inline-block'}),
        dcc.Graph(id='graph_hist', config={'displayModeBar': False},
                  style={'width': '33%', 'display': 'inline-block'}),
    ], style={'display': 'flex', 'flexDirection': 'row'}),
    
    # Controls: first row of sliders
    html.Div([
        html.Div([
            html.Label("Temperature (K)"),
            dcc.Slider(id='slider_T', min=100, max=600, step=1, value=300,
                       marks={i: str(i) for i in range(100, 601, 100)}),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        html.Div([
            html.Label("Eg (eV)"),
            dcc.Slider(id='slider_Eg', min=0.2, max=3.0, step=0.01, value=1.12,
                       marks={0.2:"0.2", 1.0:"1.0", 2.0:"2.0", 3.0:"3.0"}),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
        html.Div([
            html.Label("Bin Number"),
            dcc.Slider(id='slider_bins', min=5, max=50, step=1, value=20,
                       marks={i: str(i) for i in range(5, 51, 5)}),
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
    ]),
    # Controls: second row of sliders
    html.Div([
        html.Div([
            html.Label("P-Type Doping (log10[Na])"),
            dcc.Slider(id='slider_Na', min=14, max=19, step=1, value=14,
                       marks={i: str(i) for i in range(14, 20)}),
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '10px'}),
        html.Div([
            html.Label("N-Type Doping (log10[Nd])"),
            dcc.Slider(id='slider_Nd', min=14, max=19, step=1, value=14,
                       marks={i: str(i) for i in range(14, 20)}),
        ], style={'width': '45%', 'display': 'inline-block', 'padding': '10px'}),
    ]),
    html.Div([
        html.Label("E–f Diagram Scale:"),
        dcc.RadioItems(
            id='radio_scale',
            options=[
                {'label': 'LINEAR', 'value': 'linear'},
                {'label': 'LOG', 'value': 'log'}
            ],
            value='linear',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'}
        )
    ], style={'padding': '10px'}),
    
    dcc.Interval(id='interval-component', interval=300, n_intervals=0),
    # Hidden store to keep track of the current maximum count (for histogram x-axis)
    dcc.Store(id='store_ymax', data=0)
])

# ---------------------------
# Callback to Update All Figures and Stored y_max
# ---------------------------
@app.callback(
    [Output('graph_ex', 'figure'),
     Output('graph_ef', 'figure'),
     Output('graph_hist', 'figure'),
     Output('store_ymax', 'data')],
    [Input('interval-component', 'n_intervals'),
     Input('slider_T', 'value'),
     Input('slider_Eg', 'value'),
     Input('slider_Na', 'value'),
     Input('slider_Nd', 'value'),
     Input('radio_scale', 'value'),
     Input('slider_bins', 'value')],
    [State('store_ymax', 'data')]
)
def update_figures(n_intervals, T, Eg, Na_log, Nd_log, scale, bins, stored_ymax):
    # Calculate simulation parameters
    Vt, Na, Nd, ni, n, p, Ef = calculate_parameters(T, Eg, Na_log, Nd_log)
    aa = Eg / 2

    # Common energy range for all plots
    y_min = -aa - ww
    y_max_energy = aa + ww

    # ---------------------
    # E–x Diagram (Spatial)
    # ---------------------
    shapes_ex = [
        dict(type="rect", x0=0, y0=y_min, x1=2.8, y1=-aa,
             fillcolor="red", line=dict(color="red"), layer="below"),
        dict(type="rect", x0=0, y0=aa, x1=2.8, y1=y_max_energy,
             fillcolor="green", line=dict(color="green"), layer="below"),
        dict(type="line", x0=0, y0=Ef, x1=2.8, y1=Ef,
             line=dict(color="blue", dash="dash", width=2))
    ]
    xx_e, yy_e, xx_h, yy_h = generate_scatter_points(n, p, aa, Vt)
    scatter_traces_ex = []
    if xx_e.size > 0:
        scatter_traces_ex.append(go.Scatter(
            x=xx_e, y=yy_e, mode='markers', marker=dict(color='black', size=8),
            name='Electrons'
        ))
    if xx_h.size > 0:
        scatter_traces_ex.append(go.Scatter(
            x=xx_h, y=yy_h, mode='markers', marker=dict(color='white', size=8),
            name='Holes'
        ))
    layout_ex = go.Layout(
        xaxis=dict(range=[0, 2.8], showgrid=False, zeroline=False, title="x"),
        yaxis=dict(range=[y_min, y_max_energy], showgrid=False, zeroline=False, title="Energy (eV)"),
        shapes=shapes_ex,
        annotations=[dict(x=2.8, y=Ef, text="E_F", showarrow=False,
                          font=dict(color="blue", size=12))],
        plot_bgcolor='rgb(204,204,204)',
        margin=dict(l=20, r=20, t=20, b=20)
    )
    fig_ex = go.Figure(data=scatter_traces_ex, layout=layout_ex)

    # ---------------------
    # E–f Diagram (Energy-Probability)
    # ---------------------
    y_vals = np.linspace(y_min, y_max_energy, 400)
    f_vals = 1.0/(1 + np.exp((y_vals - Ef) / Vt))
    fh_vals = 1 - f_vals

    trace_f = go.Scatter(x=f_vals, y=y_vals, mode='lines', line=dict(width=3, color='black'),
                           name='f')
    trace_fh = go.Scatter(x=fh_vals, y=y_vals, mode='lines', line=dict(width=3, color='white'),
                            name='f_h')
    offsets = [0, 0.08, 0.15, 0.35]
    hv_lines = [go.Scatter(x=[1e-10, 1], y=[y_min - off, y_min - off],
                           mode='lines', line=dict(color='green', width=1), showlegend=False)
                for off in offsets]
    cv_lines = [go.Scatter(x=[1e-10, 1], y=[y_max_energy + off, y_max_energy + off],
                           mode='lines', line=dict(color='red', width=1), showlegend=False)
                for off in offsets]
    trace_Ef = go.Scatter(x=[1e-10, 1], y=[Ef, Ef],
                           mode='lines', line=dict(color='blue', dash='dash', width=2),
                           name='E_F')
    traces_ef = [trace_f, trace_fh, trace_Ef] + hv_lines + cv_lines
    # Change the x-axis range if log scale is selected
    if scale == 'log':
        xaxis_range = [-10, 1]
    else:
        xaxis_range = [-0.05, 1.05]
    layout_ef = go.Layout(
        xaxis=dict(type=scale, range=xaxis_range, title="Probability", autorange=False),
        yaxis=dict(range=[y_min, y_max_energy], title="Energy (eV)", autorange=False),
        plot_bgcolor='rgb(204,204,204)',
        margin=dict(l=50, r=20, t=20, b=20)
    )
    fig_ef = go.Figure(data=traces_ef, layout=layout_ef)

    # ---------------------
    # Combined Histogram (Electrons & Holes) with flipped axes
    # ---------------------
    histogram_traces = []
    counts_list = []
    if yy_e.size > 0:
        counts_e, bin_edges_e = np.histogram(yy_e, bins=bins, range=[y_min, y_max_energy])
        bin_centers_e = (bin_edges_e[:-1] + bin_edges_e[1:]) / 2
        trace_hist_e = go.Bar(y=bin_centers_e, x=counts_e, orientation='h',
                              marker=dict(color='black'),
                              name="Electrons", opacity=0.7)
        histogram_traces.append(trace_hist_e)
        counts_list.append(np.max(counts_e))
    if yy_h.size > 0:
        counts_h, bin_edges_h = np.histogram(yy_h, bins=bins, range=[y_min, y_max_energy])
        bin_centers_h = (bin_edges_h[:-1] + bin_edges_h[1:]) / 2
        trace_hist_h = go.Bar(y=bin_centers_h, x=counts_h, orientation='h',
                              marker=dict(color='white', line=dict(color='black')),
                              name="Holes", opacity=0.7)
        histogram_traces.append(trace_hist_h)
        counts_list.append(np.max(counts_h))
    
    computed_max = max(counts_list) if counts_list else 10

    # Determine which input triggered the callback; reset stored y_max on parameter change
    triggered_inputs = [t['prop_id'] for t in callback_context.triggered]
    param_changed = any(key in t for t in triggered_inputs for key in ['slider_T', 'slider_Eg', 'slider_Na', 'slider_Nd'])
    
    if param_changed or stored_ymax is None:
        new_ymax = computed_max
    else:
        new_ymax = max(stored_ymax, computed_max)
    
    layout_hist = go.Layout(
        barmode='overlay',
        xaxis=dict(title="Count", range=[0, new_ymax]),
        yaxis=dict(title="Energy (eV)", range=[y_min, y_max_energy]),
        plot_bgcolor='rgb(204,204,204)',
        margin=dict(l=50, r=20, t=20, b=20)
    )
    fig_hist = go.Figure(data=histogram_traces, layout=layout_hist)
    
    if not histogram_traces:
        fig_hist = go.Figure()
        fig_hist.update_layout(
            xaxis_title="Count",
            yaxis_title="Energy (eV)",
            plot_bgcolor='rgb(204,204,204)',
            margin=dict(l=50, r=20, t=20, b=20),
            annotations=[dict(text="No electrons or holes", x=0.5, y=0.5,
                              xref="paper", yref="paper", showarrow=False)]
        )
    
    return fig_ex, fig_ef, fig_hist, new_ymax

# ---------------------------
# Run the App in Notebook Mode
# ---------------------------
app.run_server(mode='inline', debug=True)
